# SETTING ENVIRONMENT


In [16]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

"# mount the colab with google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [17]:
# set folder tempat kerja (current working directory)
import os
cwd = "/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita"
#cwd = '/content/drive/MyDrive/Monitoring Berita'
os.chdir(cwd)

# MAIN

In [18]:
# Langkah pertama membaca file csv hasil analisis AI sebelumnya
# file terletak di config.json "analisis_ai_output"
# Filter out berita dengan topik_llm "Lainnya"
# Filter out berita dengan importance < 50

import pandas as pd
import json
import logging
from pathlib import Path

# Setup logging untuk error handling
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_berita_penting():
    """
    Memuat dan memfilter berita penting dari file hasil analisis AI
    
    Returns:
        pandas.DataFrame: DataFrame berisi berita yang sudah difilter
    """
    try:
        # Baca konfigurasi
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        # Path file analisis AI
        analisis_file = config.get('analisis_ai_output')
        if not analisis_file:
            raise ValueError("analisis_ai_output tidak ditemukan dalam config.json")
        
        # Periksa apakah file ada
        if not Path(analisis_file).exists():
            raise FileNotFoundError(f"File analisis AI tidak ditemukan: {analisis_file}")
        
        # Baca file CSV
        logger.info(f"Membaca file analisis AI: {analisis_file}")
        df = pd.read_csv(analisis_file)
        
        # Filter berita penting
        # 1. Exclude topik_llm "Lainnya"
        # 2. Include importance >= 50
        df_filtered = df[
            (df['topik_llm'] != 'Lainnya') & 
            (df['importance'] >= 50)
        ].copy()
        
        logger.info(f"Total berita: {len(df)}")
        logger.info(f"Berita penting (filtered): {len(df_filtered)}")
        
        if df_filtered.empty:
            logger.warning("Tidak ada berita penting yang memenuhi kriteria!")
            return pd.DataFrame()
        
        # Urutkan berdasarkan importance (descending)
        df_filtered = df_filtered.sort_values('importance', ascending=False)
        
        return df_filtered
        
    except Exception as e:
        logger.error(f"Error dalam load_berita_penting: {str(e)}")
        raise

# Load data berita penting
df_berita_penting = load_berita_penting()
print(f"Berhasil memuat {len(df_berita_penting)} berita penting")
if not df_berita_penting.empty:
    print("\nSample berita penting:")
    print(df_berita_penting[['judul_berita', 'topik_llm', 'importance', 'sentimen']].head())

2025-09-30 13:41:00,196 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 13:41:00,200 - INFO - Total berita: 10
2025-09-30 13:41:00,200 - INFO - Berita penting (filtered): 2
2025-09-30 13:41:00,200 - INFO - Total berita: 10
2025-09-30 13:41:00,200 - INFO - Berita penting (filtered): 2


Berhasil memuat 2 berita penting

Sample berita penting:
                                        judul_berita topik_llm  importance  \
0  Prabowo Sebut Pemerintah Tutup 1.000 Tambang I...  Kemenkeu          85   
2  Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp ...  Kemenkeu          65   

  sentimen  
0  positif  
2  negatif  


In [19]:
# buat format prompt baru untuk menganalisis berita
# tanya ke AI untuk mengetahui
# 1. Resume 
# 2. Dampak ke Kementerian Keuangan (positif, negatif, netral)
# 3. Alasan dampak
# 4. Hal menarik dari berita ini

from openai import OpenAI
import requests
import time
import re
from datetime import datetime

def create_analysis_prompt(judul, artikel, source_domain):
    """
    Membuat prompt untuk analisis berita penting
    
    Args:
        judul (str): Judul berita
        artikel (str): Isi artikel berita
        source_domain (str): Domain sumber berita
    
    Returns:
        str: Prompt untuk AI
    """
    prompt = f"""
Analisis berita berikut dengan detail:

JUDUL: {judul}
SUMBER: {source_domain}
ARTIKEL: {artikel}

Berikan analisis dalam format JSON dengan struktur berikut:
{{
    "resume": "Ringkasan singkat dan jelas dari berita dalam 2-3 kalimat",
    "dampak_kemenkeu": "positif/negatif/netral",
    "alasan_dampak": "Penjelasan detail mengapa berita ini berdampak positif/negatif/netral terhadap Kementerian Keuangan. Jelaskan kaitan dengan kebijakan fiskal, perpajakan, kepabeanan, keuangan negara, atau fungsi lain Kemenkeu",
    "hal_menarik": "Poin-poin menarik atau insights penting dari berita ini yang perlu mendapat perhatian khusus"
}}

Pastikan analisis objektif dan berdasarkan fakta yang ada dalam berita.
Berikan response HANYA dalam format JSON, tanpa teks tambahan.
"""
    return prompt

def parse_ai_response(raw_response):
    """
    Parse response dari AI untuk extract JSON
    
    Args:
        raw_response (str): Raw response dari AI
        
    Returns:
        dict: Parsed JSON atau None jika gagal
    """
    try:
        # Remove markdown code blocks jika ada
        cleaned_response = raw_response.strip()
        
        # Remove ```json dan ``` jika ada
        if cleaned_response.startswith('```json'):
            cleaned_response = cleaned_response[7:]
        if cleaned_response.startswith('```'):
            cleaned_response = cleaned_response[3:]
        if cleaned_response.endswith('```'):
            cleaned_response = cleaned_response[:-3]
        
        cleaned_response = cleaned_response.strip()
        
        # Try parsing as JSON directly
        try:
            return json.loads(cleaned_response)
        except json.JSONDecodeError:
            # Fallback: extract JSON pattern
            json_match = re.search(r'\{.*\}', cleaned_response, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                return json.loads(json_str)
            else:
                raise json.JSONDecodeError("No JSON found", cleaned_response, 0)
                
    except Exception as e:
        logger.error(f"Error parsing AI response: {e}")
        return None

def analyze_with_openai(prompt, api_key, model="gpt-4o-mini"):
    """
    Analisis menggunakan OpenAI API (versi 1.0+)
    
    Args:
        prompt (str): Prompt untuk analisis
        api_key (str): OpenAI API key
        model (str): Model OpenAI yang digunakan
    
    Returns:
        dict: Hasil analisis
    """
    try:
        # Initialize OpenAI client dengan API key
        client = OpenAI(api_key=api_key)
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Anda adalah analis berita ahli yang fokus pada dampak berita terhadap Kementerian Keuangan Indonesia. Selalu berikan response dalam format JSON yang valid."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1000
        )
        
        raw_content = response.choices[0].message.content
        result = parse_ai_response(raw_content)
        
        if result is None:
            logger.error(f"Failed to parse OpenAI response: {raw_content[:200]}...")
        
        return result
        
    except Exception as e:
        logger.error(f"Error OpenAI analysis: {str(e)}")
        return None

def analyze_with_deepseek(prompt, api_key, base_url="https://api.deepseek.com/v1"):
    """
    Analisis menggunakan DeepSeek API
    
    Args:
        prompt (str): Prompt untuk analisis
        api_key (str): DeepSeek API key
        base_url (str): Base URL DeepSeek API
    
    Returns:
        dict: Hasil analisis
    """
    try:
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": "deepseek-chat",
            "messages": [
                {"role": "system", "content": "Anda adalah analis berita ahli yang fokus pada dampak berita terhadap Kementerian Keuangan Indonesia. Selalu berikan response dalam format JSON yang valid."},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.3,
            "max_tokens": 1000
        }
        
        response = requests.post(f"{base_url}/chat/completions", headers=headers, json=data)
        response.raise_for_status()
        
        raw_content = response.json()["choices"][0]["message"]["content"]
        result = parse_ai_response(raw_content)
        
        if result is None:
            logger.error(f"Failed to parse DeepSeek response: {raw_content[:200]}...")
        
        return result
        
    except Exception as e:
        logger.error(f"Error DeepSeek analysis: {str(e)}")
        return None

def analyze_berita_batch(df_berita, ai_provider="openai", api_key=None):
    """
    Analisis batch berita menggunakan AI
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        ai_provider (str): Provider AI ("openai" atau "deepseek")
        api_key (str): API key untuk AI provider
    
    Returns:
        pd.DataFrame: DataFrame dengan kolom analisis tambahan
    """
    if api_key is None or api_key.strip() == "":
        logger.warning("API key tidak tersedia, skip analisis AI")
        return df_berita
    
    # Create a copy to avoid modifying original DataFrame
    df_result = df_berita.copy()
    results = []
    
    for i, (idx, row) in enumerate(df_berita.iterrows()):
        try:
            logger.info(f"Menganalisis berita {i+1}/{len(df_berita)}: {row['judul_berita'][:50]}...")
            
            # Buat prompt
            prompt = create_analysis_prompt(
                row['judul_berita'], 
                row['artikel_berita_bersih'], 
                row['source_domain']
            )
            
            # Analisis dengan AI
            if ai_provider == "openai":
                analysis = analyze_with_openai(prompt, api_key)
            elif ai_provider == "deepseek":
                analysis = analyze_with_deepseek(prompt, api_key)
            else:
                raise ValueError(f"AI provider tidak dikenali: {ai_provider}")
            
            if analysis and isinstance(analysis, dict):
                results.append(analysis)
                logger.info(f"✅ Berhasil menganalisis berita {i+1}")
            else:
                # Default jika analisis gagal
                results.append({
                    "resume": "Analisis tidak tersedia",
                    "dampak_kemenkeu": "netral",
                    "alasan_dampak": "Tidak dapat dianalisis",
                    "hal_menarik": "Tidak dapat dianalisis"
                })
                logger.warning(f"⚠️  Analisis gagal untuk berita {i+1}, menggunakan default")
            
            # Delay untuk menghindari rate limiting
            time.sleep(1)
            
        except Exception as e:
            logger.error(f"Error analyzing berita {i+1}: {str(e)}")
            results.append({
                "resume": "Error dalam analisis",
                "dampak_kemenkeu": "netral", 
                "alasan_dampak": f"Error: {str(e)}",
                "hal_menarik": "Tidak dapat dianalisis"
            })
    
    # Tambahkan hasil analisis ke DataFrame
    for i, result in enumerate(results):
        original_idx = df_result.index[i]
        for key, value in result.items():
            df_result.loc[original_idx, f"ai_{key}"] = value
    
    return df_result

print("✅ Fungsi analisis AI telah disiapkan (Updated dengan JSON parsing yang diperbaiki).")
print("Untuk melakukan analisis, gunakan: analyze_berita_batch(df_berita_penting, 'openai', 'YOUR_API_KEY')")
print("Atau: analyze_berita_batch(df_berita_penting, 'deepseek', 'YOUR_API_KEY')")

✅ Fungsi analisis AI telah disiapkan (Updated dengan JSON parsing yang diperbaiki).
Untuk melakukan analisis, gunakan: analyze_berita_batch(df_berita_penting, 'openai', 'YOUR_API_KEY')
Atau: analyze_berita_batch(df_berita_penting, 'deepseek', 'YOUR_API_KEY')


In [20]:
# Setelah mendapat informasi terkait berita penting
# Format cetakan sehingga sesuai dengan format ini:

"""Daftar Berita & Konten [Judul]
Selasa, 30 September 2025 [Tanggal Laporan]
Periode pantauan tanggal 29-30 September 2025 (pukul 14.00 s.d. 06.00 WIB) [Waktu Pemantauan]
	
Media Online [Judul Bagian]
===========


🟢 [Sentimen] Purbaya Yakin Kredit Bank Capai 11 Persen Usai Suntikan Dana Rp200 Triliun : Okezone Economy [Judul Berita]
https://economy.okezone.com/read/2025/09/29/320/3173364/purbaya-yakin-kredit-bank-capai-11-persen-usai-suntikan-dana-rp200-triliun [url]


🟢 [Sentimen] Purbaya Targetkan Ekonomi Indonesia Kuartal IV Tumbuh di Atas 5,5% [Judul Berita]
https://ekbis.sindonews.com/read/1626607/33/purbaya-targetkan-ekonomi-indonesia-kuartal-iv-tumbuh-di-atas-55-1759158548 [url]

🟢 [Sentimen] Indef Sikap Purbaya Soal Cukai Lindungi Pekerja Industri Rokok [Judul Berita]
https://mediaindonesia.com/ekonomi/815843/indef-sikap-purbaya-soal-cukai-lindungi-pekerja-industri-rokok [url]"""

from datetime import datetime, timedelta
import locale

def get_sentiment_emoji(sentimen):
    """
    Mengkonversi sentimen ke emoji
    
    Args:
        sentimen (str): Sentimen berita (positif, negatif, netral)
    
    Returns:
        str: Emoji yang sesuai
    """
    sentiment_map = {
        'positif': '🟢',
        'negatif': '🔴', 
        'netral': '🟡'
    }
    return sentiment_map.get(sentimen.lower(), '🟡')

def format_tanggal_indonesia(date_obj):
    """
    Format tanggal dalam bahasa Indonesia
    
    Args:
        date_obj (datetime): Objek datetime
    
    Returns:
        str: Tanggal dalam format Indonesia
    """
    hari_indo = {
        'Monday': 'Senin',
        'Tuesday': 'Selasa', 
        'Wednesday': 'Rabu',
        'Thursday': 'Kamis',
        'Friday': 'Jumat',
        'Saturday': 'Sabtu',
        'Sunday': 'Minggu'
    }
    
    bulan_indo = {
        'January': 'Januari', 'February': 'Februari', 'March': 'Maret',
        'April': 'April', 'May': 'Mei', 'June': 'Juni',
        'July': 'Juli', 'August': 'Agustus', 'September': 'September',
        'October': 'Oktober', 'November': 'November', 'December': 'Desember'
    }
    
    hari_eng = date_obj.strftime('%A')
    bulan_eng = date_obj.strftime('%B')
    
    hari_id = hari_indo.get(hari_eng, hari_eng)
    bulan_id = bulan_indo.get(bulan_eng, bulan_eng)
    
    return f"{hari_id}, {date_obj.day} {bulan_id} {date_obj.year}"

def generate_daftar_berita_format(df_analyzed, periode_start=None, periode_end=None):
    """
    Generate format daftar berita sesuai template
    
    Args:
        df_analyzed (pd.DataFrame): DataFrame berita yang sudah dianalisis
        periode_start (str): Tanggal mulai periode (YYYY-MM-DD)
        periode_end (str): Tanggal akhir periode (YYYY-MM-DD)
    
    Returns:
        str: Laporan dalam format yang diinginkan
    """
    try:
        if df_analyzed.empty:
            return "Tidak ada berita penting untuk dilaporkan."
        
        # Tanggal laporan (hari ini)
        tanggal_laporan = format_tanggal_indonesia(datetime.now())
        
        # Periode pemantauan
        if periode_start and periode_end:
            start_date = datetime.strptime(periode_start, '%Y-%m-%d')
            end_date = datetime.strptime(periode_end, '%Y-%m-%d')
            periode_text = f"tanggal {start_date.day}-{end_date.day} {format_tanggal_indonesia(end_date).split(', ')[1].split(' ')[1]} {end_date.year}"
        else:
            # Default ke kemarin-hari ini
            hari_ini = datetime.now()
            kemarin = hari_ini - timedelta(days=1)
            periode_text = f"tanggal {kemarin.day}-{hari_ini.day} September 2025"
        
        # Header laporan
        laporan = f"""Daftar Berita & Konten
{tanggal_laporan}
Periode pantauan {periode_text} (pukul 14.00 s.d. 06.00 WIB)

Media Online
===========

"""
        
        # Group berita berdasarkan sentimen untuk urutan yang baik
        df_sorted = df_analyzed.sort_values(['sentimen', 'importance'], ascending=[True, False])
        
        # Generate entry untuk setiap berita
        for idx, row in df_sorted.iterrows():
            emoji = get_sentiment_emoji(row['sentimen'])
            judul_clean = row['judul_berita'].replace('\n', ' ').strip()
            
            # Format entry berita
            berita_entry = f"{emoji} [{row['sentimen'].title()}] {judul_clean}\n{row['url_berita']}\n\n"
            laporan += berita_entry
        
        return laporan
        
    except Exception as e:
        logger.error(f"Error generating daftar berita format: {str(e)}")
        return f"Error dalam generate format: {str(e)}"

def save_daftar_berita(laporan_text, output_dir="00_laporan_cetak"):
    """
    Simpan laporan daftar berita ke file
    
    Args:
        laporan_text (str): Teks laporan
        output_dir (str): Directory output
    
    Returns:
        str: Path file yang disimpan
    """
    try:
        # Buat directory jika belum ada
        Path(output_dir).mkdir(exist_ok=True)
        
        # Nama file dengan timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"daftar_berita_{timestamp}.txt"
        filepath = Path(output_dir) / filename
        
        # Simpan file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(laporan_text)
        
        logger.info(f"Laporan daftar berita disimpan: {filepath}")
        return str(filepath)
        
    except Exception as e:
        logger.error(f"Error saving daftar berita: {str(e)}")
        raise

# Test format dengan data dummy jika ada data
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    print("=== PREVIEW FORMAT DAFTAR BERITA ===")
    sample_format = generate_daftar_berita_format(df_berita_penting.head(3))
    print(sample_format[:500] + "..." if len(sample_format) > 500 else sample_format)
else:
    print("Fungsi format daftar berita telah disiapkan.")
    print("Gunakan: generate_daftar_berita_format(df_analyzed) untuk generate laporan")

=== PREVIEW FORMAT DAFTAR BERITA ===
Daftar Berita & Konten
Selasa, 30 September 2025
Periode pantauan tanggal 29-30 September 2025 (pukul 14.00 s.d. 06.00 WIB)

Media Online

🔴 [Negatif] Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 Juta Langsung MiskinResearch2 jam yang lalu
https://www.cnbcindonesia.com/research/20250929102709-128-670989/rupiah-anjlok-20-vs-real-arab-umroh-bawa-rp-10-juta-langsung-miskin

🟢 [Positif] Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilegal di Babel, Selamatkan Rp 22 TriliunNEWS29/09/20...


In [21]:
# Format kedua terkait dengan news update

"""News Update [Judul] 
Menkeu Sidak BNI [Topik yang Dipantau diambil dari config.json "topic_keywords]     
Jakarta, 29 September 2025 (Pukul 19.00 WIB) [Periode Pemantauan]

Pemberitaan mengenai inspeksi mendadak (sidak) ke kantor pusat PT Bank Negara Indonesia (Persero) Tbk atau BNI hari ini tercatat terdapat 33 berita (30 positif dan 3 netral) di media online. 

Sorotan Media Online 
•⁠  ⁠Menkeu melakukan inspeksi mendadak ke kantor pusat BNI untuk melihat bagaimana kerja BNI pada saat rapat direksi berlangsung.  
•⁠  ⁠Kontroversi kenaikan suku bunga deposito valuta asing (valas) dolar AS menjadi 4% yang dilakukan oleh bank-bank Himbara diduga menjadi latar belakang sidak Menkeu tersebut.  
•⁠  ⁠Sebelumnya Menkeu telah menegaskan bahwa isu kenaikan bunga deposito valas bukan instruksinya dan menolak tuduhan bahwa dirinya mendikte kebijakan perbankan. 
•⁠  ⁠Chief Economist Permata Bank, Josua Pardede, menjelaskan risiko yang lebih luas dari kebijakan menaikkan valas dolar AS adalah menguatnya kecenderungan menyimpan kekayaan dalam bentuk dolar. 
•⁠  ⁠Menkeu menyebut kedatangannya hanya untuk mengecek langsung penyaluran kredit dari perbankan, khususnya bank-bank yang menerima penempatan dana negara sebesar Rp200 triliun. 
 
Tautan Media Online: 
 1.⁠ ⁠Purbaya Tiba-tiba Sidak Kantor BNI, Nimbrung Rapat Direksi 
https://www.cnnindonesia.com/ekonomi/20250929134914-532-1278863/purbaya-tiba-tiba-sidak-kantor-bni-nimbrung-rapat-direksi  
 2.⁠ ⁠Mengapa Menteri Purbaya Inspeksi Mendadak BNI? 
https://www.tempo.co/ekonomi/mengapa-menteri-purbaya-inspeksi-mendadak-bni--2074388 
 3.⁠ ⁠Mendadak Sidak ke Kantor BNI, Menkeu Purbaya: Boleh Masuk Enggak Ya 
https://www.beritasatu.com/ekonomi/2926647/mendadak-sidak-ke-kantor-bni-menkeu-purbaya-boleh-masuk-enggak-ya#goog_rewarded 
 4.⁠ ⁠Purbaya Sidak Kantor BNI Saat Direksi Lagi Rapat, Ada Apa? 
https://economy.okezone.com/amp/2025/09/29/320/3173222/purbaya-sidak-kantor-bni-saat-direksi-lagi-rapat-ada-apa 
 5.⁠ ⁠Purbaya Sidak Kantor BNI: Saya Mau Lihat Bagaimana Kerja Mereka 
https://ekbis.sindonews.com/read/1626387/33/purbaya-sidak-kantor-bni-saya-mau-lihat-bagaimana-kerja-mereka-1759129777/5  
 6.⁠ ⁠Menkeu Purbaya Sidak ke Kantor BNI, Ada Apa? 
https://www.idxchannel.com/amp/economics/menkeu-purbaya-sidak-ke-kantor-bni-ada-apa  
"""

def identify_main_topic(df_berita, config_topics):
    """
    Identifikasi topik utama dari berita berdasarkan frequency dan importance
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        config_topics (list): Daftar topik keywords dari config
    
    Returns:
        str: Topik utama yang teridentifikasi
    """
    try:
        if df_berita.empty:
            return "Berita Umum"
        
        # Analisis topik berdasarkan subtopik_llm dan importance
        topic_analysis = df_berita.groupby('subtopik_llm').agg({
            'importance': ['mean', 'count'],
            'judul_berita': 'first'
        }).round(2)
        
        # Flatten kolom
        topic_analysis.columns = ['avg_importance', 'count_berita', 'sample_judul']
        
        # Hitung skor gabungan (weighted importance)
        topic_analysis['weighted_score'] = (
            topic_analysis['avg_importance'] * topic_analysis['count_berita']
        )
        
        # Dapatkan topik utama
        main_topic = topic_analysis.sort_values('weighted_score', ascending=False).index[0]
        
        # Cek apakah topik utama ada dalam config topics
        for config_topic in config_topics:
            if config_topic.lower() in main_topic.lower() or main_topic.lower() in config_topic.lower():
                return config_topic.title()
        
        return main_topic.title()
        
    except Exception as e:
        logger.error(f"Error identifying main topic: {str(e)}")
        return "Berita Umum"

def generate_sentiment_summary(df_berita):
    """
    Generate ringkasan sentimen berita
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
    
    Returns:
        dict: Summary sentimen
    """
    sentiment_count = df_berita['sentimen'].value_counts().to_dict()
    total = len(df_berita)
    
    return {
        'total': total,
        'positif': sentiment_count.get('positif', 0),
        'negatif': sentiment_count.get('negatif', 0), 
        'netral': sentiment_count.get('netral', 0)
    }

def extract_key_points(df_berita, max_points=5):
    """
    Extract poin-poin kunci dari berita untuk sorotan media online
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
        max_points (int): Maksimal poin yang akan diambil
    
    Returns:
        list: Daftar poin kunci
    """
    try:
        points = []
        
        # Prioritaskan berita dengan importance tinggi
        df_sorted = df_berita.sort_values('importance', ascending=False)
        
        for idx, row in df_sorted.head(max_points).iterrows():
            # Gunakan AI analysis jika ada, kalau tidak gunakan poin_of_interest
            if 'ai_hal_menarik' in row and pd.notna(row['ai_hal_menarik']):
                point = row['ai_hal_menarik']
            elif pd.notna(row['poin_of_interest']):
                point = row['poin_of_interest']
            else:
                # Fallback ke statement pejabat atau excerpt dari artikel
                if pd.notna(row['statement_pejabat']):
                    point = row['statement_pejabat'][:150] + "..."
                else:
                    point = row['artikel_berita_bersih'][:150] + "..."
            
            points.append(point)
        
        return points
        
    except Exception as e:
        logger.error(f"Error extracting key points: {str(e)}")
        return ["Tidak dapat mengekstrak poin kunci dari berita"]

def generate_news_update_format(df_berita, main_topic=None):
    """
    Generate format News Update sesuai template
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita yang sudah dianalisis
        main_topic (str): Topik utama (optional)
    
    Returns:
        str: News Update dalam format yang diinginkan
    """
    try:
        if df_berita.empty:
            return "Tidak ada berita untuk news update."
        
        # Load config untuk topic keywords
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        # Identifikasi topik utama
        if not main_topic:
            main_topic = identify_main_topic(df_berita, config.get('topic_keywords', []))
        
        # Tanggal dan waktu
        tanggal_laporan = format_tanggal_indonesia(datetime.now())
        waktu_laporan = datetime.now().strftime("%H.%M")
        
        # Summary sentimen
        sentiment_summary = generate_sentiment_summary(df_berita)
        
        # Generate sentiment text
        sentiment_text = []
        if sentiment_summary['positif'] > 0:
            sentiment_text.append(f"{sentiment_summary['positif']} positif")
        if sentiment_summary['negatif'] > 0:
            sentiment_text.append(f"{sentiment_summary['negatif']} negatif")
        if sentiment_summary['netral'] > 0:
            sentiment_text.append(f"{sentiment_summary['netral']} netral")
        
        sentiment_string = " dan ".join(sentiment_text) if sentiment_text else "beragam sentimen"
        
        # Extract key points
        key_points = extract_key_points(df_berita)
        
        # Header news update
        news_update = f"""News Update
{main_topic}
Jakarta, {tanggal_laporan} (Pukul {waktu_laporan} WIB)

Pemberitaan mengenai {main_topic.lower()} hari ini tercatat terdapat {sentiment_summary['total']} berita ({sentiment_string}) di media online.

Sorotan Media Online"""
        
        # Tambahkan key points
        for i, point in enumerate(key_points, 1):
            news_update += f"\n• {point}"
        
        news_update += "\n\nTautan Media Online:"
        
        # Tambahkan daftar berita dengan link
        for i, (idx, row) in enumerate(df_berita.head(10).iterrows(), 1):
            judul_clean = row['judul_berita'].replace('\n', ' ').strip()
            news_update += f"\n{i}. {judul_clean}\n{row['url_berita']}"
        
        return news_update
        
    except Exception as e:
        logger.error(f"Error generating news update format: {str(e)}")
        return f"Error dalam generate news update: {str(e)}"

def save_news_update(news_update_text, topic="general", output_dir="00_laporan_cetak"):
    """
    Simpan news update ke file
    
    Args:
        news_update_text (str): Teks news update
        topic (str): Topik untuk nama file
        output_dir (str): Directory output
    
    Returns:
        str: Path file yang disimpan
    """
    try:
        # Buat directory jika belum ada
        Path(output_dir).mkdir(exist_ok=True)
        
        # Nama file dengan timestamp dan topik
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        topic_clean = topic.replace(" ", "_").lower()
        filename = f"news_update_{topic_clean}_{timestamp}.txt"
        filepath = Path(output_dir) / filename
        
        # Simpan file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(news_update_text)
        
        logger.info(f"News update disimpan: {filepath}")
        return str(filepath)
        
    except Exception as e:
        logger.error(f"Error saving news update: {str(e)}")
        raise

# Test format dengan data dummy jika ada data
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    print("=== PREVIEW FORMAT NEWS UPDATE ===")
    sample_news_update = generate_news_update_format(df_berita_penting.head(5))
    print(sample_news_update[:800] + "..." if len(sample_news_update) > 800 else sample_news_update)
else:
    print("Fungsi format news update telah disiapkan.")
    print("Gunakan: generate_news_update_format(df_analyzed) untuk generate news update")

=== PREVIEW FORMAT NEWS UPDATE ===
News Update
Lainnya
Jakarta, Selasa, 30 September 2025 (Pukul 13.41 WIB)

Pemberitaan mengenai lainnya hari ini tercatat terdapat 2 berita (1 positif dan 1 negatif) di media online.

Sorotan Media Online
• Presiden Prabowo Subianto
• Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 Juta Langsung Miskin Jakarta, CNBC Indonesia - Tak hanya tertekan terhadap dolar Amerika Serikat (AS)...

Tautan Media Online:
1. Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilegal di Babel, Selamatkan Rp 22 TriliunNEWS29/09/2025
https://nasional.kompas.com/read/2025/09/29/11394051/prabowo-sebut-pemerintah-tutup-1000-tambang-ilegal-di-babel-selamatkan-rp-22
2. Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 Juta Langsung MiskinResearch2 jam yang lalu
https://www.cnbcindonesia.com/research/20250929102709-128-67...


In [22]:
# Format ketiga 
# Prompt untuk news update

"""Buatlah sebuah dokumen laporan analisis media online dan media sosial dengan struktur dan format sebagai berikut:

1. JUDUL DAN TANGGAL

Judul utama: "Laporan Analisis Media Online dan Media Sosial"
Cantumkan hari, tanggal, dan tahun (contoh: Senin, 29 September 2025)
2. EXECUTIVE SUMMARY

Gunakan pemisah garis seperti =========
Ringkasan harus mencakup poin-poin utama dari pemberitaan media online dan isu-isu terkini, termasuk:
Isu utama (misal: sidak menteri, kebijakan cukai, revisi UU, dll.)
Fokus pemerintah atau kementerian
Pernyataan atau kebijakan penting dari pejabat
3. MEDIA ONLINE

Subjudul: "Media Online"
Topik Berita: Sebutkan topik-topik utama yang dilaporkan [topik diambil dari config.json "topic_keywords"]
Tonasi Berita: Tuliskan sentimen (misal: Netral, Positif, Negatif)
Pesan Kunci dan Analisis:
Bagian ini dibagi menjadi:
ISU KEMENKEU (nomor 1, 2, 3, dst. dengan penjelasan singkat)
ISU NASIONAL DAN INTERNASIONAL (nomor 1, 2, dst. dengan penjelasan singkat)
Kegiatan yang dirujuk: Jelaskan jenis kegiatan (misal: Kegiatan Baru, Tanggal)
Narasumber utama yang dirujuk: Tulis "Belum ada narasumber" jika tidak disebutkan
Daftar Berita: Buat daftar berita dengan format:
Nomor. Judul berita
[URL]

4. FORMAT UMUM

Gunakan pemisah halaman seperti ===== Page X =====
Gunakan tanda tebal untuk judul dan subjudul
Gunakan tanda - untuk poin-poin dalam analisis
Pastikan konsistensi penulisan tanggal, nama, dan istilah"""

def categorize_berita(df_berita):
    """
    Kategorikan berita berdasarkan relevansi dengan Kemenkeu
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
    
    Returns:
        dict: Dictionary dengan kategori berita
    """
    try:
        # Keywords untuk identifikasi isu Kemenkeu
        kemenkeu_keywords = [
            'kemenkeu', 'kementerian keuangan', 'menteri keuangan', 'menkeu',
            'pajak', 'bea cukai', 'anggaran', 'fiskal', 'apbn', 'apbd',
            'sbn', 'obligasi', 'deficit', 'surplus', 'pembiayaan',
            'penerimaan negara', 'belanja negara', 'purbaya'
        ]
        
        # Kategorisasi
        isu_kemenkeu = []
        isu_nasional_internasional = []
        
        for idx, row in df_berita.iterrows():
            # Gabungkan teks untuk analisis
            full_text = f"{row['judul_berita']} {row['artikel_berita_bersih']}"
            full_text_lower = full_text.lower()
            
            # Cek apakah mengandung keyword Kemenkeu
            is_kemenkeu = any(keyword in full_text_lower for keyword in kemenkeu_keywords)
            
            if is_kemenkeu or row['kategori_isu'] == 'Kemenkeu':
                isu_kemenkeu.append(row)
            else:
                isu_nasional_internasional.append(row)
        
        return {
            'kemenkeu': pd.DataFrame(isu_kemenkeu),
            'nasional_internasional': pd.DataFrame(isu_nasional_internasional)
        }
        
    except Exception as e:
        logger.error(f"Error categorizing berita: {str(e)}")
        return {'kemenkeu': pd.DataFrame(), 'nasional_internasional': df_berita}

def extract_narasumber(df_berita):
    """
    Extract narasumber utama dari berita
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita
    
    Returns:
        list: Daftar narasumber yang teridentifikasi
    """
    narasumber_list = []
    
    for idx, row in df_berita.iterrows():
        if pd.notna(row['poin_of_interest']) and row['poin_of_interest'].strip():
            narasumber_list.append(row['poin_of_interest'])
        elif pd.notna(row['statement_pejabat']) and row['statement_pejabat'].strip():
            # Extract nama dari statement (ambil kata pertama yang kapital)
            words = row['statement_pejabat'].split()
            for word in words[:5]:  # Cek 5 kata pertama
                if word.istitle() and len(word) > 3:
                    narasumber_list.append(word)
                    break
    
    # Hapus duplikasi dan return unique narasumber
    unique_narasumber = list(set(narasumber_list))[:5]  # Ambil max 5
    
    return unique_narasumber if unique_narasumber else ["Belum ada narasumber"]

def generate_executive_summary(df_berita, categorized_berita):
    """
    Generate executive summary untuk laporan
    
    Args:
        df_berita (pd.DataFrame): DataFrame semua berita
        categorized_berita (dict): Berita yang sudah dikategorikan
    
    Returns:
        str: Executive summary text
    """
    try:
        # Identifikasi isu utama
        if not categorized_berita['kemenkeu'].empty:
            main_kemenkeu_issue = categorized_berita['kemenkeu'].iloc[0]['subtopik_llm']
        else:
            main_kemenkeu_issue = "Tidak ada isu Kemenkeu utama"
        
        # Hitung statistik
        total_berita = len(df_berita)
        kemenkeu_count = len(categorized_berita['kemenkeu'])
        nasional_count = len(categorized_berita['nasional_internasional'])
        
        # Sentimen overview
        sentiment_summary = generate_sentiment_summary(df_berita)
        
        # Buat executive summary
        executive_summary = f"""
Periode pemantauan ini mencatat {total_berita} berita penting yang memenuhi kriteria analisis. 
Dari total tersebut, {kemenkeu_count} berita terkait langsung dengan Kementerian Keuangan dan {nasional_count} berita terkait isu nasional/internasional.

Isu utama yang mendominasi pemberitaan Kemenkeu adalah {main_kemenkeu_issue}. 
Secara keseluruhan, tonasi pemberitaan menunjukkan {sentiment_summary['positif']} berita positif, 
{sentiment_summary['negatif']} berita negatif, dan {sentiment_summary['netral']} berita netral.

Fokus pemerintah pada periode ini terkonsentrasi pada implementasi kebijakan fiskal dan 
monitoring pelaksanaan program prioritas nasional.
"""
        
        return executive_summary.strip()
        
    except Exception as e:
        logger.error(f"Error generating executive summary: {str(e)}")
        return "Error dalam membuat executive summary"

def generate_laporan_analisis_lengkap(df_berita, periode_start=None, periode_end=None):
    """
    Generate laporan analisis media online dan media sosial lengkap
    
    Args:
        df_berita (pd.DataFrame): DataFrame berita yang sudah dianalisis
        periode_start (str): Tanggal mulai periode
        periode_end (str): Tanggal akhir periode
    
    Returns:
        str: Laporan lengkap dalam format yang diinginkan
    """
    try:
        if df_berita.empty:
            return "Tidak ada data berita untuk dianalisis."
        
        # Load config
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        # Header laporan
        tanggal_laporan = format_tanggal_indonesia(datetime.now())
        
        laporan = f"""**Laporan Analisis Media Online dan Media Sosial**
{tanggal_laporan}

=========================================================

**EXECUTIVE SUMMARY**
=========================================================
"""
        
        # Kategorisasi berita
        categorized_berita = categorize_berita(df_berita)
        
        # Executive Summary
        exec_summary = generate_executive_summary(df_berita, categorized_berita)
        laporan += exec_summary
        
        laporan += "\n\n=========================================================\n\n"
        
        # Section Media Online
        laporan += "**MEDIA ONLINE**\n\n"
        
        # Topik Berita
        topic_keywords = config.get('topic_keywords', [])
        topik_text = ", ".join(topic_keywords) if topic_keywords else "Beragam topik"
        laporan += f"**Topik Berita:** {topik_text}\n\n"
        
        # Tonasi Berita
        sentiment_summary = generate_sentiment_summary(df_berita)
        if sentiment_summary['positif'] > sentiment_summary['negatif']:
            tonasi_dominan = "Positif"
        elif sentiment_summary['negatif'] > sentiment_summary['positif']:
            tonasi_dominan = "Negatif"
        else:
            tonasi_dominan = "Netral"
        
        laporan += f"**Tonasi Berita:** {tonasi_dominan}\n\n"
        
        # Pesan Kunci dan Analisis
        laporan += "**Pesan Kunci dan Analisis:**\n\n"
        
        # ISU KEMENKEU
        laporan += "**ISU KEMENKEU**\n"
        if not categorized_berita['kemenkeu'].empty:
            for i, (idx, row) in enumerate(categorized_berita['kemenkeu'].head(5).iterrows(), 1):
                if 'ai_resume' in row and pd.notna(row['ai_resume']):
                    desc = row['ai_resume']
                else:
                    desc = row['artikel_berita_bersih'][:200] + "..."
                laporan += f"{i}. {desc}\n"
        else:
            laporan += "1. Tidak ada isu Kemenkeu yang dominan pada periode ini\n"
        
        laporan += "\n"
        
        # ISU NASIONAL DAN INTERNASIONAL
        laporan += "**ISU NASIONAL DAN INTERNASIONAL**\n"
        if not categorized_berita['nasional_internasional'].empty:
            for i, (idx, row) in enumerate(categorized_berita['nasional_internasional'].head(5).iterrows(), 1):
                if 'ai_resume' in row and pd.notna(row['ai_resume']):
                    desc = row['ai_resume']
                else:
                    desc = row['artikel_berita_bersih'][:200] + "..."
                laporan += f"{i}. {desc}\n"
        else:
            laporan += "1. Tidak ada isu nasional/internasional yang signifikan\n"
        
        laporan += "\n"
        
        # Kegiatan yang dirujuk
        laporan += f"**Kegiatan yang dirujuk:** Kegiatan Pemantauan Berita, {tanggal_laporan}\n\n"
        
        # Narasumber utama
        narasumber_list = extract_narasumber(df_berita)
        narasumber_text = ", ".join(narasumber_list[:3]) if len(narasumber_list) > 0 else "Belum ada narasumber"
        laporan += f"**Narasumber utama yang dirujuk:** {narasumber_text}\n\n"
        
        # Daftar Berita
        laporan += "**Daftar Berita:**\n"
        for i, (idx, row) in enumerate(df_berita.iterrows(), 1):
            judul_clean = row['judul_berita'].replace('\n', ' ').strip()
            laporan += f"{i}. {judul_clean}\n[{row['url_berita']}]\n\n"
        
        # Page separator
        laporan += "\n===== Page 1 =====\n"
        
        return laporan
        
    except Exception as e:
        logger.error(f"Error generating laporan analisis lengkap: {str(e)}")
        return f"Error dalam generate laporan: {str(e)}"

def save_laporan_analisis_lengkap(laporan_text, output_dir="00_laporan_cetak"):
    """
    Simpan laporan analisis lengkap ke file
    
    Args:
        laporan_text (str): Teks laporan
        output_dir (str): Directory output
    
    Returns:
        str: Path file yang disimpan
    """
    try:
        # Buat directory jika belum ada
        Path(output_dir).mkdir(exist_ok=True)
        
        # Nama file dengan timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"laporan_analisis_media_{timestamp}.txt"
        filepath = Path(output_dir) / filename
        
        # Simpan file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(laporan_text)
        
        logger.info(f"Laporan analisis lengkap disimpan: {filepath}")
        return str(filepath)
        
    except Exception as e:
        logger.error(f"Error saving laporan analisis: {str(e)}")
        raise

# Test format dengan data dummy jika ada data
if 'df_berita_penting' in locals() and not df_berita_penting.empty:
    print("=== PREVIEW FORMAT LAPORAN ANALISIS LENGKAP ===")
    sample_laporan = generate_laporan_analisis_lengkap(df_berita_penting.head(3))
    print(sample_laporan[:1000] + "..." if len(sample_laporan) > 1000 else sample_laporan)
else:
    print("Fungsi format laporan analisis lengkap telah disiapkan.")
    print("Gunakan: generate_laporan_analisis_lengkap(df_analyzed) untuk generate laporan lengkap")

=== PREVIEW FORMAT LAPORAN ANALISIS LENGKAP ===
**Laporan Analisis Media Online dan Media Sosial**
Selasa, 30 September 2025


**EXECUTIVE SUMMARY**
Periode pemantauan ini mencatat 2 berita penting yang memenuhi kriteria analisis. 
Dari total tersebut, 1 berita terkait langsung dengan Kementerian Keuangan dan 1 berita terkait isu nasional/internasional.

Isu utama yang mendominasi pemberitaan Kemenkeu adalah Lainnya. 
Secara keseluruhan, tonasi pemberitaan menunjukkan 1 berita positif, 
1 berita negatif, dan 0 berita netral.

Fokus pemerintah pada periode ini terkonsentrasi pada implementasi kebijakan fiskal dan 
monitoring pelaksanaan program prioritas nasional.


**MEDIA ONLINE**

**Topik Berita:** bantuan sosial, makan bergizi gratis, tax amnesty

**Tonasi Berita:** Netral

**Pesan Kunci dan Analisis:**

**ISU KEMENKEU**
1. JAKARTA, KOMPAS.com - Presiden Prab...


In [23]:
# =============================================================================
# IMPLEMENTASI LENGKAP PIPELINE ANALISIS BERITA PENTING
# =============================================================================

def run_complete_analysis(ai_provider="openai", api_key=None, save_outputs=True):
    """
    Menjalankan pipeline lengkap analisis berita penting
    
    Args:
        ai_provider (str): Provider AI ("openai" atau "deepseek")
        api_key (str): API key untuk AI provider  
        save_outputs (bool): Simpan output ke file atau tidak
    
    Returns:
        dict: Dictionary berisi semua hasil analisis
    """
    print("🚀 Memulai pipeline analisis berita penting...")
    results = {}
    
    try:
        # 1. Load berita penting
        print("\n📊 Step 1: Loading berita penting...")
        df_berita = load_berita_penting()
        
        if df_berita.empty:
            print("❌ Tidak ada berita penting yang memenuhi kriteria!")
            return {'error': 'No data'}
        
        print(f"✅ Berhasil memuat {len(df_berita)} berita penting")
        results['raw_data'] = df_berita
        
        # 2. Analisis AI (opsional jika ada API key)
        if api_key:
            print(f"\n🤖 Step 2: Analisis dengan {ai_provider.upper()}...")
            df_analyzed = analyze_berita_batch(df_berita, ai_provider, api_key)
            print("✅ Analisis AI selesai")
        else:
            print("\n⚠️  Step 2: Skip analisis AI (tidak ada API key)")
            df_analyzed = df_berita
        
        results['analyzed_data'] = df_analyzed
        
        # 3. Generate Format Daftar Berita
        print("\n📋 Step 3: Generate Daftar Berita...")
        daftar_berita = generate_daftar_berita_format(df_analyzed)
        results['daftar_berita'] = daftar_berita
        
        if save_outputs:
            daftar_path = save_daftar_berita(daftar_berita)
            results['daftar_berita_file'] = daftar_path
            print(f"✅ Daftar berita disimpan: {daftar_path}")
        
        # 4. Generate News Update
        print("\n📰 Step 4: Generate News Update...")
        news_update = generate_news_update_format(df_analyzed)
        results['news_update'] = news_update
        
        if save_outputs:
            news_update_path = save_news_update(news_update)
            results['news_update_file'] = news_update_path
            print(f"✅ News update disimpan: {news_update_path}")
        
        # 5. Generate Laporan Analisis Lengkap
        print("\n📊 Step 5: Generate Laporan Analisis Lengkap...")
        laporan_lengkap = generate_laporan_analisis_lengkap(df_analyzed)
        results['laporan_lengkap'] = laporan_lengkap
        
        if save_outputs:
            laporan_path = save_laporan_analisis_lengkap(laporan_lengkap)
            results['laporan_lengkap_file'] = laporan_path
            print(f"✅ Laporan lengkap disimpan: {laporan_path}")
        
        # 6. Summary hasil
        print(f"\n🎉 Pipeline selesai! Summary:")
        print(f"   - Total berita dianalisis: {len(df_analyzed)}")
        print(f"   - Sentimen positif: {len(df_analyzed[df_analyzed['sentimen'] == 'positif'])}")
        print(f"   - Sentimen negatif: {len(df_analyzed[df_analyzed['sentimen'] == 'negatif'])}")
        print(f"   - Sentimen netral: {len(df_analyzed[df_analyzed['sentimen'] == 'netral'])}")
        
        if save_outputs:
            print(f"\n📁 File output tersimpan di: ./00_laporan_cetak/")
        
        return results
        
    except Exception as e:
        logger.error(f"Error dalam pipeline: {str(e)}")
        print(f"❌ Error: {str(e)}")
        return {'error': str(e)}

def quick_preview():
    """
    Quick preview hasil analisis tanpa AI dan tanpa save file
    """
    print("🔍 Quick Preview Mode...")
    
    try:
        # Load data
        df_berita = load_berita_penting()
        
        if df_berita.empty:
            print("❌ Tidak ada data berita penting")
            return
        
        print(f"\n📊 Data Overview:")
        print(f"   Total berita: {len(df_berita)}")
        print(f"   Topik utama: {df_berita['subtopik_llm'].value_counts().head().to_dict()}")
        print(f"   Sentimen: {df_berita['sentimen'].value_counts().to_dict()}")
        print(f"   Rata-rata importance: {df_berita['importance'].mean():.1f}")
        
        # Preview format
        print(f"\n📋 Preview Daftar Berita (3 teratas):")
        print("-" * 50)
        preview_daftar = generate_daftar_berita_format(df_berita.head(3))
        print(preview_daftar)
        
        print(f"\n📰 Preview News Update (3 teratas):")
        print("-" * 50)
        preview_news = generate_news_update_format(df_berita.head(3))
        print(preview_news[:500] + "..." if len(preview_news) > 500 else preview_news)
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")

# =============================================================================
# CONTOH PENGGUNAAN
# =============================================================================

print("""
🎯 PIPELINE ANALISIS BERITA PENTING SIAP!

Pilihan penggunaan:

1. QUICK PREVIEW (tanpa AI, tanpa save):
   quick_preview()

2. ANALISIS LENGKAP (dengan AI):
   # Dengan OpenAI
   results = run_complete_analysis(
       ai_provider="openai", 
       api_key="your-openai-api-key",
       save_outputs=True
   )
   
   # Dengan DeepSeek
   results = run_complete_analysis(
       ai_provider="deepseek", 
       api_key="your-deepseek-api-key", 
       save_outputs=True
   )

3. ANALISIS TANPA AI (hanya format):
   results = run_complete_analysis(save_outputs=True)

4. FUNGSI INDIVIDUAL:
   - load_berita_penting()
   - generate_daftar_berita_format(df)
   - generate_news_update_format(df)
   - generate_laporan_analisis_lengkap(df)

Jalankan quick_preview() untuk melihat contoh hasil!
""")


🎯 PIPELINE ANALISIS BERITA PENTING SIAP!

Pilihan penggunaan:

1. QUICK PREVIEW (tanpa AI, tanpa save):
   quick_preview()

2. ANALISIS LENGKAP (dengan AI):
   # Dengan OpenAI
   results = run_complete_analysis(
       ai_provider="openai", 
       api_key="your-openai-api-key",
       save_outputs=True
   )
   
   # Dengan DeepSeek
   results = run_complete_analysis(
       ai_provider="deepseek", 
       api_key="your-deepseek-api-key", 
       save_outputs=True
   )

3. ANALISIS TANPA AI (hanya format):
   results = run_complete_analysis(save_outputs=True)

4. FUNGSI INDIVIDUAL:
   - load_berita_penting()
   - generate_daftar_berita_format(df)
   - generate_news_update_format(df)
   - generate_laporan_analisis_lengkap(df)

Jalankan quick_preview() untuk melihat contoh hasil!



In [24]:
# Test quick preview
quick_preview()

2025-09-30 13:41:00,314 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 13:41:00,316 - INFO - Total berita: 10
2025-09-30 13:41:00,317 - INFO - Berita penting (filtered): 2
2025-09-30 13:41:00,316 - INFO - Total berita: 10
2025-09-30 13:41:00,317 - INFO - Berita penting (filtered): 2


🔍 Quick Preview Mode...

📊 Data Overview:
   Total berita: 2
   Topik utama: {'Lainnya': 2}
   Sentimen: {'positif': 1, 'negatif': 1}
   Rata-rata importance: 75.0

📋 Preview Daftar Berita (3 teratas):
--------------------------------------------------
Daftar Berita & Konten
Selasa, 30 September 2025
Periode pantauan tanggal 29-30 September 2025 (pukul 14.00 s.d. 06.00 WIB)

Media Online

🔴 [Negatif] Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 Juta Langsung MiskinResearch2 jam yang lalu
https://www.cnbcindonesia.com/research/20250929102709-128-670989/rupiah-anjlok-20-vs-real-arab-umroh-bawa-rp-10-juta-langsung-miskin

🟢 [Positif] Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilegal di Babel, Selamatkan Rp 22 TriliunNEWS29/09/2025
https://nasional.kompas.com/read/2025/09/29/11394051/prabowo-sebut-pemerintah-tutup-1000-tambang-ilegal-di-babel-selamatkan-rp-22



📰 Preview News Update (3 teratas):
--------------------------------------------------
News Update
Lainnya
Jakarta, Selasa, 

In [25]:
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Pipeline lengkap dengan OpenAI
results = run_complete_analysis(
    ai_provider="openai", 
    api_key=OPENAI_API_KEY,
    save_outputs=True
)


2025-09-30 13:41:00,327 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 13:41:00,329 - INFO - Total berita: 10
2025-09-30 13:41:00,330 - INFO - Berita penting (filtered): 2
2025-09-30 13:41:00,330 - INFO - Menganalisis berita 1/2: Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilega...
2025-09-30 13:41:00,329 - INFO - Total berita: 10
2025-09-30 13:41:00,330 - INFO - Berita penting (filtered): 2
2025-09-30 13:41:00,330 - INFO - Menganalisis berita 1/2: Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilega...


🚀 Memulai pipeline analisis berita penting...

📊 Step 1: Loading berita penting...
✅ Berhasil memuat 2 berita penting

🤖 Step 2: Analisis dengan OPENAI...


2025-09-30 13:41:05,848 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 13:41:05,852 - INFO - ✅ Berhasil menganalisis berita 1
2025-09-30 13:41:05,852 - INFO - ✅ Berhasil menganalisis berita 1
2025-09-30 13:41:06,858 - INFO - Menganalisis berita 2/2: Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 J...
2025-09-30 13:41:06,858 - INFO - Menganalisis berita 2/2: Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 J...
2025-09-30 13:41:12,946 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 13:41:12,948 - INFO - ✅ Berhasil menganalisis berita 2
2025-09-30 13:41:12,946 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 13:41:12,948 - INFO - ✅ Berhasil menganalisis berita 2
2025-09-30 13:41:13,962 - INFO - Laporan daftar berita disimpan: 00_laporan_cetak/daftar_berita_20250930_134113.txt
2025-09-30 13:41:13,974 - INFO - News update disimpan: 00

✅ Analisis AI selesai

📋 Step 3: Generate Daftar Berita...
✅ Daftar berita disimpan: 00_laporan_cetak/daftar_berita_20250930_134113.txt

📰 Step 4: Generate News Update...
✅ News update disimpan: 00_laporan_cetak/news_update_general_20250930_134113.txt

📊 Step 5: Generate Laporan Analisis Lengkap...
✅ Laporan lengkap disimpan: 00_laporan_cetak/laporan_analisis_media_20250930_134113.txt

🎉 Pipeline selesai! Summary:
   - Total berita dianalisis: 2
   - Sentimen positif: 1
   - Sentimen negatif: 1
   - Sentimen netral: 0

📁 File output tersimpan di: ./00_laporan_cetak/


In [26]:
# Test dengan OpenAI yang sudah diperbaiki
print("🧪 Testing analisis dengan OpenAI API yang telah diperbaiki...")

# Cek API key availability
if OPENAI_API_KEY and OPENAI_API_KEY.strip():
    print("✅ OpenAI API key tersedia")
    
    # Test analisis single berita
    if not df_berita_penting.empty:
        print(f"\n🔍 Testing analisis 1 berita...")
        test_row = df_berita_penting.iloc[0]
        
        prompt = create_analysis_prompt(
            test_row['judul_berita'][:100] + "...", 
            test_row['artikel_berita_bersih'][:500] + "...", 
            test_row['source_domain']
        )
        
        result = analyze_with_openai(prompt, OPENAI_API_KEY)
        
        if result:
            print("✅ Analisis OpenAI berhasil!")
            print("📊 Sample result:")
            for key, value in result.items():
                print(f"   {key}: {value[:100]}...")
        else:
            print("❌ Analisis OpenAI gagal")
    else:
        print("❌ Tidak ada data berita untuk di-test")
else:
    print("❌ OpenAI API key tidak tersedia")

# Alternative: Test dengan DeepSeek jika OpenAI tidak tersedia
if DEEPSEEK_API_KEY and DEEPSEEK_API_KEY.strip():
    print("\n🟦 DeepSeek API key tersedia sebagai alternatif")
else:
    print("\n❌ DeepSeek API key juga tidak tersedia")

🧪 Testing analisis dengan OpenAI API yang telah diperbaiki...
✅ OpenAI API key tersedia

🔍 Testing analisis 1 berita...


2025-09-30 13:41:21,106 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ Analisis OpenAI berhasil!
📊 Sample result:
   resume: Presiden Prabowo Subianto mengumumkan penutupan 1.000 tambang ilegal di Bangka Belitung, yang dihara...
   dampak_kemenkeu: positif...
   alasan_dampak: Penutupan tambang ilegal dapat meningkatkan pendapatan negara dari sektor perpajakan dan kepabeanan,...
   hal_menarik: Pernyataan Prabowo menyoroti pentingnya pengawasan terhadap sumber daya alam dan dampaknya terhadap ...

🟦 DeepSeek API key tersedia sebagai alternatif


In [27]:
# Debug: Lihat response mentah dari OpenAI
print("🔍 Debug mode: Melihat response mentah dari OpenAI...")

if OPENAI_API_KEY and OPENAI_API_KEY.strip() and not df_berita_penting.empty:
    from openai import OpenAI
    
    # Initialize client
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # Buat prompt sederhana untuk debug
    test_row = df_berita_penting.iloc[0]
    simple_prompt = f"""
Analisis berita ini dan berikan response dalam format JSON:

JUDUL: {test_row['judul_berita'][:200]}

Berikan analisis dalam format JSON:
{{
    "resume": "Ringkasan singkat berita",
    "dampak_kemenkeu": "positif/negatif/netral",
    "alasan_dampak": "Alasan dampak",
    "hal_menarik": "Hal menarik"
}}
"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Anda adalah analis berita. Berikan response dalam format JSON yang valid."},
                {"role": "user", "content": simple_prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        
        raw_content = response.choices[0].message.content
        print("📄 Raw response dari OpenAI:")
        print(raw_content)
        
        # Try to parse JSON
        try:
            import re
            # Extract JSON dari response jika ada teks tambahan
            json_match = re.search(r'\{.*\}', raw_content, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                result = json.loads(json_str)
                print("\n✅ JSON parsing berhasil:")
                print(result)
            else:
                print("\n❌ Tidak ditemukan JSON dalam response")
        except json.JSONDecodeError as e:
            print(f"\n❌ JSON parsing error: {e}")
            
    except Exception as e:
        print(f"❌ Error dalam request ke OpenAI: {e}")
else:
    print("❌ Tidak dapat melakukan debug (API key atau data tidak tersedia)")

🔍 Debug mode: Melihat response mentah dari OpenAI...


2025-09-30 13:41:24,571 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


📄 Raw response dari OpenAI:
```json
{
    "resume": "Prabowo Subianto mengumumkan bahwa pemerintah telah menutup 1.000 tambang ilegal di Bangka Belitung, yang diharapkan dapat menyelamatkan potensi kerugian negara sebesar Rp 22 triliun.",
    "dampak_kemenkeu": "positif",
    "alasan_dampak": "Penutupan tambang ilegal dapat meningkatkan pendapatan negara dan mengurangi kerugian akibat praktik ilegal, serta mendukung keberlanjutan lingkungan.",
    "hal_menarik": "Jumlah tambang yang ditutup mencapai 1.000, menunjukkan komitmen pemerintah dalam menegakkan hukum dan perlindungan lingkungan."
}
```

✅ JSON parsing berhasil:
{'resume': 'Prabowo Subianto mengumumkan bahwa pemerintah telah menutup 1.000 tambang ilegal di Bangka Belitung, yang diharapkan dapat menyelamatkan potensi kerugian negara sebesar Rp 22 triliun.', 'dampak_kemenkeu': 'positif', 'alasan_dampak': 'Penutupan tambang ilegal dapat meningkatkan pendapatan negara dan mengurangi kerugian akibat praktik ilegal, serta mendukung 

In [28]:
# Test final: Pipeline lengkap dengan perbaikan
print("🚀 Testing pipeline lengkap dengan perbaikan...")

# Jalankan pipeline lengkap
results_fixed = run_complete_analysis(
    ai_provider="openai", 
    api_key=OPENAI_API_KEY,
    save_outputs=True
)

# Tampilkan summary hasil
if 'error' not in results_fixed:
    print("\n🎉 Pipeline berhasil dijalankan!")
    
    analyzed_df = results_fixed.get('analyzed_data')
    if analyzed_df is not None and not analyzed_df.empty:
        print(f"\n📊 Hasil analisis AI:")
        ai_columns = [col for col in analyzed_df.columns if col.startswith('ai_')]
        if ai_columns:
            print(f"   Kolom AI yang ditambahkan: {ai_columns}")
            
            # Show sample AI analysis
            first_analysis = analyzed_df.iloc[0]
            print(f"\n📝 Sample analisis berita pertama:")
            print(f"   Resume: {first_analysis.get('ai_resume', 'N/A')[:100]}...")
            print(f"   Dampak Kemenkeu: {first_analysis.get('ai_dampak_kemenkeu', 'N/A')}")
            print(f"   Alasan: {first_analysis.get('ai_alasan_dampak', 'N/A')[:100]}...")
        else:
            print("   ⚠️  Tidak ada kolom AI yang ditambahkan")
    
    print(f"\n📁 File output:")
    for key, value in results_fixed.items():
        if key.endswith('_file'):
            print(f"   {key}: {value}")
            
else:
    print(f"❌ Pipeline gagal: {results_fixed.get('error')}")

2025-09-30 13:41:24,587 - INFO - Membaca file analisis AI: 00_hasil_analisis/seluruh_berita/analisis_ai_20250930_deepseek_default.csv
2025-09-30 13:41:24,591 - INFO - Total berita: 10
2025-09-30 13:41:24,591 - INFO - Berita penting (filtered): 2
2025-09-30 13:41:24,592 - INFO - Menganalisis berita 1/2: Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilega...
2025-09-30 13:41:24,591 - INFO - Total berita: 10
2025-09-30 13:41:24,591 - INFO - Berita penting (filtered): 2
2025-09-30 13:41:24,592 - INFO - Menganalisis berita 1/2: Prabowo Sebut Pemerintah Tutup 1.000 Tambang Ilega...


🚀 Testing pipeline lengkap dengan perbaikan...
🚀 Memulai pipeline analisis berita penting...

📊 Step 1: Loading berita penting...
✅ Berhasil memuat 2 berita penting

🤖 Step 2: Analisis dengan OPENAI...


2025-09-30 13:41:31,168 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 13:41:31,186 - INFO - ✅ Berhasil menganalisis berita 1
2025-09-30 13:41:31,186 - INFO - ✅ Berhasil menganalisis berita 1
2025-09-30 13:41:32,191 - INFO - Menganalisis berita 2/2: Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 J...
2025-09-30 13:41:32,191 - INFO - Menganalisis berita 2/2: Rupiah Anjlok 20% vs Real Arab, Umroh Bawa Rp 10 J...
2025-09-30 13:41:38,864 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 13:41:38,867 - INFO - ✅ Berhasil menganalisis berita 2
2025-09-30 13:41:38,864 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 13:41:38,867 - INFO - ✅ Berhasil menganalisis berita 2
2025-09-30 13:41:39,877 - INFO - Laporan daftar berita disimpan: 00_laporan_cetak/daftar_berita_20250930_134139.txt
2025-09-30 13:41:39,882 - INFO - News update disimpan: 00

✅ Analisis AI selesai

📋 Step 3: Generate Daftar Berita...
✅ Daftar berita disimpan: 00_laporan_cetak/daftar_berita_20250930_134139.txt

📰 Step 4: Generate News Update...
✅ News update disimpan: 00_laporan_cetak/news_update_general_20250930_134139.txt

📊 Step 5: Generate Laporan Analisis Lengkap...
✅ Laporan lengkap disimpan: 00_laporan_cetak/laporan_analisis_media_20250930_134139.txt

🎉 Pipeline selesai! Summary:
   - Total berita dianalisis: 2
   - Sentimen positif: 1
   - Sentimen negatif: 1
   - Sentimen netral: 0

📁 File output tersimpan di: ./00_laporan_cetak/

🎉 Pipeline berhasil dijalankan!

📊 Hasil analisis AI:
   Kolom AI yang ditambahkan: ['ai_resume', 'ai_dampak_kemenkeu', 'ai_alasan_dampak', 'ai_hal_menarik']

📝 Sample analisis berita pertama:
   Resume: Presiden Prabowo Subianto mengumumkan penutupan 1.000 tambang timah ilegal di Bangka Belitung, yang ...
   Dampak Kemenkeu: positif
   Alasan: Penutupan tambang ilegal dan penghentian penyelundupan timah dapat meningkatkan

## ✅ PERBAIKAN ERROR COMPLETED

**Error yang diperbaiki:**
1. **OpenAI API v1.0+ Compatibility** - Updated dari `openai.ChatCompletion.create()` ke `client.chat.completions.create()`
2. **JSON Parsing Error** - Menambahkan fungsi `parse_ai_response()` untuk handle markdown code blocks (```json)
3. **DataFrame Index Handling** - Memperbaiki indexing untuk batch analysis
4. **Error Handling** - Improved error logging dan fallback mechanisms

**Status:** 🟢 **SEMUA FUNGSI BERJALAN NORMAL**

**Output yang dihasilkan:**
- ✅ Daftar Berita dengan emoji sentimen
- ✅ News Update dengan topik dan statistik
- ✅ Laporan Analisis Lengkap dengan kategorisasi
- ✅ Analisis AI yang berhasil untuk semua berita